<img src="https://electronaut.info/wp-content/uploads/2017/05/Tubescape_jupyter2.jpg">

<h1><font color = b30000>TUBE DATA MATCHER for uTRACER FILES</font></h1><h5>Anode current for a sweep of grid voltages  when anode voltage = constant</h5>


# STEP 1 — Import the data

In [68]:
chosen_folder = input("Please enter the path to the folder containing the uTracer files: ")
print("PLEASE NOTE: This script assumes all the files in the chosen directory are for the same type of tube!\n")        
tube_type = input("What type of tubes are these?")

print("\n— — — — — \n")

import os 
os.chdir(chosen_folder)
list_of_tube_data_files_in_dir = []
for file in os.listdir(chosen_folder):
    if file.endswith(".utd"):
        list_of_tube_data_files_in_dir.append(file)

print("• This batch consists of", len(list_of_tube_data_files_in_dir), "type", tube_type, "tubes. The filenames are:")
print("\n", list_of_tube_data_files_in_dir)

Please enter the path to the folder containing the uTracer files: /Users/rrmc/github_rrMacKinnon/Electrona_uTracer/SampleTubeData
PLEASE NOTE: This script assumes all the files in the chosen directory are for the same type of tube!

What type of tubes are these?5749

— — — — — 

• This batch consists of 49 type 5749 tubes. The filenames are:

 ['1.utd', '10.utd', '11.utd', '12.utd', '13.utd', '14.utd', '15.utd', '16.utd', '17.utd', '18.utd', '19.utd', '2.utd', '20.utd', '21.utd', '22.utd', '23.utd', '24.utd', '25.utd', '26.utd', '27.utd', '28.utd', '29.utd', '3.utd', '30.utd', '31.utd', '32.utd', '33.utd', '34.utd', '35.utd', '36.utd', '37.utd', '38.utd', '39.utd', '4.utd', '40.utd', '42.utd', '43.utd', '44.utd', '45.utd', '46.utd', '47.utd', '48.utd', '49.utd', '5.utd', '50.utd', '6.utd', '7.utd', '8.utd', '9.utd']


# STEP 2 — Process the uTracer files and build a dataframe

In [75]:
import pandas as pd

batch_list = list_of_tube_data_files_in_dir
master_tube_list = []

def import_tube_data_file(file_to_open):
    while True:
        try:
            with open(file_to_open, 'r') as file:
                file_contents_string = str(file.read())
                file_contents_list = file_contents_string.split()

                #  Extract the data from the file_contents_list and build a dictionary
                x_values = [i for i in file_contents_list[8::2]]
                y_values = [i for i in file_contents_list[9::2]]
                anode_voltage = {'anode_voltage': file_contents_list[6]}
                tube_ID = {'tube_ID': file_to_open[:-4]}
                tube_data_extras = {**tube_ID, **anode_voltage}  # Combine dicts (Python 3.5)
                tube_data_dict = {**tube_data_extras, **(dict(zip(x_values, y_values)))}

        finally:
                return tube_data_dict

for tube in batch_list:
    tube_data_dict = import_tube_data_file(tube)
    tube_data_dict.update({'tube_type':tube_type})
    master_tube_list.append(tube_data_dict) 

# Build the dataframe
df = pd.DataFrame(master_tube_list)    

# Make a list of all the columns
cols = df.columns.tolist()

# Make two new lists, separating the voltage values from the other strings.
# Also, convert the voltages to absolute value ints, so they can be numerically sorted 
voltages = []
other_stuff = []
for i in cols:
    if i.startswith('-'):
        i = int(i[1:])
        voltages.append(i)
    else:
        other_stuff.append(i)

# Sort the voltages numerically
voltages.sort()

# Make a new list to hold the voltages and the other_stuff, properly ordered.
cols_sorted = []

# Add the '-' sign back in to all the voltages and convert to strings, now that they're sorted.
voltages_sorted = []
for i in voltages:
    i = '-' + str(i)
    voltages_sorted.append(i)
    
# Re-order the other_stuff to the desired order    
other_stuff[0], other_stuff[1], other_stuff[2] = other_stuff[1], other_stuff[2], other_stuff[0]    

# Stitch the two separate lists together to form the cols_sorted list.
cols_sorted = other_stuff + voltages_sorted  

# Redefine df with the sorted column list
df = df[cols_sorted]

# Change the bias voltage columns names to something friendlier
column_names = [i.replace('-', 'Bias_') for i in cols_sorted]
df.columns = column_names

# Report how many tubes were processed
total_tubes = df.shape[0]
print("A dataframe contraining all", total_tubes, "was successfully created.")

A dataframe contraining all 49 was successfully created.


In [76]:
df.head()

,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,1,5749,325,99.278,65.383,44.717,34.082,26.669,20.931,16.492,12.992,10.291,8.23,6.669,5.495,4.626
1,10,5749,325,84.662,55.752,37.5,27.666,20.873,15.84,12.088,9.304,7.261,5.807,4.747,3.945,3.344
2,11,5749,325,57.956,39.019,26.465,19.832,14.986,11.21,8.358,6.284,4.834,3.863,3.22,2.752,2.418
3,12,5749,325,85.883,52.717,33.817,24.6,18.367,13.795,10.481,8.071,6.345,5.144,4.268,3.592,3.043
4,13,5749,325,90.664,57.346,37.541,27.686,21.015,16.003,12.239,9.433,7.395,5.967,4.977,4.267,3.759


In [87]:
# Make a new dataframe showing the statistics of the tube data
df_stats = df.describe()

# Make a bias_list containing only columns we care to see.  (We don't need stats on the tube_type or tube_ID, etc.)
bias_list = []
for i in column_names:
    if i.startswith('Bias') == True:
        bias_list.append(i)

# Display only the columns in the bias_list        
df_stats = df_stats[bias_list]

# Display the data as object so 'count' isn't a float with a million decimal places
# df_stats.astype(object)

df_stats


,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
count,49,49,49,49,49,49,49,49,49,49,49,49,49
unique,49,48,48,48,49,49,49,48,48,49,48,49,48
top,86.052,52.717,40.708,26.051,24.108,15.942,11.325,9.304,6.104,7.903,6.309,2.525,3.175
freq,1,2,2,2,1,1,1,2,2,1,2,1,2


In [80]:
# Apply a function to a column
df.Bias_2.max()

'99.278'

In [67]:
# Applymap is useful for applying a function to all data
# For example, converting all data from floats to ints
df.loc[2:7, 'Bias_2': 'Bias_50'].applymap(int)

ValueError: ("invalid literal for int() with base 10: '57.956'", 'occurred at index Bias_2')

In [49]:
# TODO:  The match-finding algorithms should remove the matched sets from the master dataframe as it computes subsequent
# TODO:  matched sets.  Otherwise, subsequent sets may contain previously paired-tubes and will be invalid.


def find_best_matched_set(tubes_df):

    # Build a list of all the tubes in the dataframe.
    tube_list = tubes_df['tube_ID'].tolist()

    # Get number of tubes to match
    n_matches = input("How many matched tubes would you like in the set?")

    # Create a list to store the master match list sets
    master_match_list = []

    # Create a dataframe to store the scores of the tube sets by primary tube_ID
    score_df = pd.DataFrame()



    for tube in tube_list:
        temp_df = tubes_df.copy()
        tube_to_match = temp_df.loc[str(str(tube))]
        bias_list = list(temp_df)[3:]  # The list of bias voltage columns;  i.e. -50, -46, -42, -38, -etc.
        bias_diff_list = []            # The list of new _diff^2 columns, to hold the difference values

        # For every bias voltage column in the DataFrame, add a new column with the suffix '_diff^2' after the name.
        for i in bias_list:
            newcolumn = str(i + '_diff^2')
            temp_df[newcolumn] = temp_df[i]
            bias_diff_list.append(newcolumn)

        # Compute the square of the difference of each current measurement for every tube, compared to the tube_to_match
        # Put all the data into the appropriate _diff^2 column.
        for i in range(len(bias_diff_list)):
            temp_df[bias_diff_list[i]] = temp_df[bias_diff_list[i]].apply(lambda x: abs(x - tube_to_match[i+3])**2)

        # Add a column called squares_sum and compute the sum of all the diff squares on each row
        temp_df['squares_sum'] = temp_df[bias_diff_list].sum(axis=1)

        # Sort ascending by squares_sum.
        temp_df_sorted = temp_df.sort_values('squares_sum')

        # Calculate the sum of all the diff squares to be used as an overall score
        tube_set_score = temp_df_sorted['squares_sum'].iloc[0:(int(n_matches)+1)].sum()
        # print("Total least-squares score for the matched tube set:", tube_set_score)
        score_df.set_value(tube, 'tube_id', tube)
        score_df.set_value(tube, 'score', tube_set_score)
        score_df_sorted = score_df.sort_values('score')
    print("\nLeast-Squares Matching Score (lower is better):")
    print(score_df_sorted)

find_best_matched_set(df)

How many matched tubes would you like in the set?8


KeyError: 'the label [1] is not in the [index]'

In [2]:
# List all available magics
% lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%perl  %%prun  %%pypy  %%python  %%python2  %%python3

In [44]:
# Display environmental variables
env_variables = %env
current_path = env_variables.get('PATH')
current_path

'/Users/rrmc/miniconda3/bin:/Users/rrmc/miniconda3/bin:/Library/Frameworks/Python.framework/Versions/3.6/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin'

In [6]:
mydict = % env
mydict.keys()

dict_keys(['TERM_PROGRAM', 'SHELL', 'TERM', 'TMPDIR', 'Apple_PubSub_Socket_Render', 'TERM_PROGRAM_VERSION', 'TERM_SESSION_ID', 'USER', 'SSH_AUTH_SOCK', '__CF_USER_TEXT_ENCODING', 'PATH', 'PWD', 'LANG', 'XPC_FLAGS', 'XPC_SERVICE_NAME', 'SHLVL', 'HOME', 'LOGNAME', 'SECURITYSESSIONID', '_', 'OLDPWD', 'JPY_PARENT_PID', 'CLICOLOR', 'PAGER', 'GIT_PAGER', 'MPLBACKEND'])

In [7]:
% env

{'Apple_PubSub_Socket_Render': '/private/tmp/com.apple.launchd.KpLw2sBcKE/Render',
 'CLICOLOR': '1',
 'GIT_PAGER': 'cat',
 'HOME': '/Users/rrmc',
 'JPY_PARENT_PID': '3478',
 'LANG': 'en_US.UTF-8',
 'LOGNAME': 'rrmc',
 'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
 'OLDPWD': '/Users/rrmc/github_rrMacKinnon',
 'PAGER': 'cat',
 'PATH': '/Users/rrmc/miniconda3/bin:/Users/rrmc/miniconda3/bin:/Library/Frameworks/Python.framework/Versions/3.6/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin',
 'PWD': '/Users/rrmc/github_rrMacKinnon/Electrona_uTracer',
 'SECURITYSESSIONID': '186a5',
 'SHELL': '/bin/bash',
 'SHLVL': '1',
 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.ipXAnwjJfW/Listeners',
 'TERM': 'xterm-color',
 'TERM_PROGRAM': 'Apple_Terminal',
 'TERM_PROGRAM_VERSION': '388.1',
 'TERM_SESSION_ID': '4ED1B075-FF11-484E-8DC5-80541C92FF99',
 'TMPDIR': '/var/folders/z8/64wcqsyn253df_sw4kb6j3s00000gn/T/',
 'USER': 'rrmc',
 'XPC_FLAGS': '0x0',
 'XPC_SERVICE_NAME': '0',
 '_': '/Users/r

In [12]:
# % time x = range(10000)
% timeit x = range(10000)
max(x)

1000000 loops, best of 3: 346 ns per loop


9999